In [1]:
import xlsxwriter
from xlsxwriter import Workbook
import cx_Oracle
import datetime
from datetime import date
import pandas as pd
import time

In [2]:
query_2 ="""select 
       k.CMS_ID, 
       Sum(k.cashback_amount) as cashback_amount,
       trunc(Sum(k.cashback_amount)) rounded_cashback_amount
  from cms.f_cashback_transaction k
 where lower(TERMOWNER) like '%test%'
       and k.bank_date between to_date('01.09.2022','dd.mm.yyyy') and to_date('01.02.2023','dd.mm.yyyy')
       and cms_id in
               (select z.cms_id
                  from cms.d_customer z
                 where z.cms_id in (select m.credit_cms_id
                                      from cms.f_transaction m
                                     where m.debitentcode = '3'
                                       and m.credit_cms_id in
                                           (select k.cms_id
                                              from cms.d_card k
                                             where k.ministry_code = '037'
                                               and k.product_code = '3')
                                       and m.value_date between '30-Jan-23' and '31-Jan-23'))
 group by k.CMS_ID
 order by 2 desc"""

In [3]:
query = """select x.client_id,
       y.NAME,
       y.FULL_NAME,
       listagg(unique x.DEBIT_PRODUCT, ' , ') x_DEBIT_PRODUCT,   
       listagg(unique x.DEBIT_TRANS_ID, ' , ') x_DEBIT_TRANS_ID,
       count(unique case when x.bank_date between '01-Nov-22' and '30-Nov-22' then x.op_id end) as November, 
       count(unique case when x.bank_date between '01-Dec-22' and '31-Dec-22' then x.op_id end) as December,
       count(unique case when x.bank_date between '01-Jan-23' and '31-Jan-23' then x.op_id end) as January,
       count(unique x.op_id) as total_say

  from birbank.f_operation x
 inner join birbank.d_customers_new y
    on x.CLIENT_ID = y.USER_ID
 where x.source_type = 3
   and x.destination_type = 2
   and x.op_status = 1
   and x.client_id in
       (select user_id
          from birbank.d_customers_new
         where cms_id in
               (select z.cms_id
                  from cms.d_customer z
                 where z.cms_id in (select m.credit_cms_id
                                      from cms.f_transaction m
                                     where m.debitentcode = '3'
                                       and m.credit_cms_id in
                                           (select k.cms_id
                                              from cms.d_card k
                                             where k.ministry_code = '037'
                                               and k.product_code = '3')
                                       and m.value_date between '30-Jan-23' and '31-Jan-23')))
   and x.bank_date between '01-Nov-22' and '31-Jan-23'  
 group by x.client_id, y.NAME, y.FULL_NAME
 having count(unique case when x.bank_date between '01-Jan-23' and '31-Jan-23' then x.op_id  end) >0
 order by TOTAL_SAY desc """

In [4]:
dsn = cx_Oracle.makedsn("dwh-scan.kapitalbank.az", 1521, service_name="report")

connection = cx_Oracle.connect(user='SAMADOVIAZ_15121[FRAUD]',password='Dwh02##3FF85A092E0EA35974',dsn=dsn)

df=pd.read_sql(query,con=connection)

df

,CLIENT_ID,NAME,FULL_NAME,X_DEBIT_PRODUCT,X_DEBIT_TRANS_ID,NOVEMBER,DECEMBER,JANUARY,TOTAL_SAY
0,386180,HEYDƏRLİ AYTAC KAMAL QIZI,None,"409858******5734 , 409858******9687 , 416973**...","5782935497 , 5797571139 , 5797599649 , 5826892...",4,10,9,23
1,220343,MUSTAFAYEV ELNUR İLHAM OĞLU,None,417211******3383,"5731948644 , 5731955375 , 5732000703 , 5732013...",12,1,3,16
2,32389,MUSAYEV RAQIF SHAKIR,None,"416973******2922 , 416973******9200 , 416974**...","5733571621 , 5968828460 , 5968984503 , 6037129...",1,3,3,7
3,211106,ELYAZOV ILHAM MUSTAFA,None,"409858******9248 , 412721******3281 , 416973**...","5763749860 , 5763779930 , 5982720583 , 6085044...",2,3,2,7
4,1907322,ISMAYILOVA GUTU NARIMAN,None,"416973******0237 , 417358******6362 , 510306**...","5898726470 , 5898769086 , 5899562351 , 6220236...",0,3,4,7
...,...,...,...,...,...,...,...,...,...
81,3553909,ƏHMƏDOV CAVAD SABİR OĞLU,None,676867******2211,6172897279,0,0,1,1
82,3769882,MAHMUDOV TEYMUR ƏKBƏR OĞLU,None,409858******7825,6183618416,0,0,1,1
83,4009895,QULIYEV MAHAMMAD SHAKIR,None,418980******1444,6183949387,0,0,1,1
84,4035232,SAFARALIYEV IBRAHIM MAMMAD,None,417358******2443,6287953514,0,0,1,1


In [5]:
%%time
dsn = cx_Oracle.makedsn("dwh-scan.kapitalbank.az", 1521, service_name="report")

connection = cx_Oracle.connect(user='SAMADOVIAZ_15121[FRAUD]',password='Dwh02##3FF85A092E0EA35974',dsn=dsn)

df=pd.read_sql(query_2,con=connection)

df

Wall time: 12min 12s


,CMS_ID,CASHBACK_AMOUNT,ROUNDED_CASHBACK_AMOUNT
0,5033833,54.0000,54
1,6730208,13.6370,13
2,4607509,11.1744,11
3,6839650,6.0773,6
4,4363501,5.3400,5
5,8447673,4.4010,4
6,4498278,4.2049,4
7,6274229,3.6640,3
8,5311365,2.7872,2
9,6209814,1.2606,1


In [6]:
%%time
dsn = cx_Oracle.makedsn("dwh-scan.kapitalbank.az", 1521, service_name="report")

connection = cx_Oracle.connect(user='SAMADOVIAZ_15121[FRAUD]',password='Dwh02##3FF85A092E0EA35974',dsn=dsn)


Wall time: 414 ms


In [7]:
query_3 ="""select *
  from cms.f_transaction_details t
 where t.mcc in (7800, 7801, 7802, 7995)
   and t.idclient in
       (select user_id
          from birbank.d_customers_new
         where cms_id in
               (select z.cms_id
                  from cms.d_customer z
                 where z.cms_id in (select m.credit_cms_id
                                      from cms.f_transaction m
                                     where m.debitentcode = '3'
                                       and m.credit_cms_id in
                                           (select k.cms_id
                                              from cms.d_card k
                                             where k.ministry_code = '037'
                                               and k.product_code = '3')
                                       and m.value_date between '30-Jan-23' and '31-Jan-23'))) """

In [8]:
%%time
df_3=pd.read_sql(query_3,con=connection)
df_3.head()

Wall time: 29min 27s


,LOAD_DATE,BANK_DATE,DOC_NO,VALUE_DATE,EXP_DATE,DEVICE,ENTCODE,ENTTYPE,ENTTYPEEXT,IDCLIENT,...,APPLE_NFC,APPLE_TOKEN_CODE,LCY_AMOUNT,TOKEN_PAN,TOKEN_TREQ_ID,TOKEN_TIME,ORIGINAL_INVOICE,DOC_NO_RVRS,FCY_AMOUNT_RVRS,LCY_AMOUNT_RVRS
0,2021-01-20,2021-01-19,2466347153,2021-01-18,NaT,P,30,F,None,424332,...,None,None,2.99,None,None,None,None,None,None,None
1,2021-02-27,2021-02-26,2645778170,2021-02-25,2023-06-30,P,30,F,None,9983,...,None,None,0.10,None,None,None,None,None,None,None
2,2021-02-27,2021-02-26,2645778175,2021-02-25,2023-06-30,P,30,F,None,9983,...,None,None,0.05,None,None,None,None,None,None,None
3,2021-04-15,2021-04-14,2789570319,2021-02-25,2023-06-30,P,30,R,None,9983,...,None,None,0.10,None,None,None,None,None,None,None
4,2021-04-15,2021-04-14,2789570321,2021-02-25,2023-06-30,P,30,R,None,9983,...,None,None,0.05,None,None,None,None,None,None,None


In [9]:
df_3

,LOAD_DATE,BANK_DATE,DOC_NO,VALUE_DATE,EXP_DATE,DEVICE,ENTCODE,ENTTYPE,ENTTYPEEXT,IDCLIENT,...,APPLE_NFC,APPLE_TOKEN_CODE,LCY_AMOUNT,TOKEN_PAN,TOKEN_TREQ_ID,TOKEN_TIME,ORIGINAL_INVOICE,DOC_NO_RVRS,FCY_AMOUNT_RVRS,LCY_AMOUNT_RVRS
0,2021-01-20,2021-01-19,2466347153,2021-01-18,NaT,P,30,F,None,424332,...,None,None,2.99,None,None,None,None,None,None,None
1,2021-02-27,2021-02-26,2645778170,2021-02-25,2023-06-30,P,30,F,None,9983,...,None,None,0.10,None,None,None,None,None,None,None
2,2021-02-27,2021-02-26,2645778175,2021-02-25,2023-06-30,P,30,F,None,9983,...,None,None,0.05,None,None,None,None,None,None,None
3,2021-04-15,2021-04-14,2789570319,2021-02-25,2023-06-30,P,30,R,None,9983,...,None,None,0.10,None,None,None,None,None,None,None
4,2021-04-15,2021-04-14,2789570321,2021-02-25,2023-06-30,P,30,R,None,9983,...,None,None,0.05,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,2023-02-23,2023-02-22,4963109658,2023-02-21,2025-08-31,P,10,S,None,4497321,...,None,None,250.00,None,None,None,None,None,None,None
1292,2023-02-23,2023-02-22,4960388354,2023-02-21,NaT,P,33,S,None,4497321,...,None,None,200.00,None,None,None,12147146,None,None,None
1293,2015-11-07,2014-07-07,428426250,2014-07-02,NaT,P,30,None,None,2747695,...,None,None,NaN,None,None,None,None,None,None,None
1294,2015-11-07,2014-07-07,428426292,2014-07-02,NaT,P,30,None,None,2747695,...,None,None,NaN,None,None,None,None,None,None,None


In [10]:
query_norton ="""
select /* +parallel*/
 a.full_name,
 a.amount,
 a.description,
 a.other_reciever,
 a.document_seria,
 a.Document_Serial_No,
 b.workflowinstance_name,
 b.startdate,
 b.startdate_time,
 B.INITIATOR,
 b.branch,
 c.user_name,
 a.wfid
  from elma.f_norton_payments a
 inner join elma.f_workflow_instance b    on a.wfid = b.wfid
 inner join elma.d_user c    on b.initiator = c.front_id
 where b.startdate > '01-Jan-23' """

In [11]:
%%time
df_norton=pd.read_sql(query_norton,con=connection)
df_norton.head()

DatabaseError: ORA-00040: active time limit exceeded - call aborted